# 나무위키 덤프 + 포켓몬 크롤링 데이터 Kor2Vec입니다.

In [1]:
import numpy as np
import tensorflow as tf
import collections
from konlpy.tag import Twitter
import re
import math
import random
import pandas as pd

DATA_PATH = "./data/"

In [2]:
def build_dataset(train_text, min_count, sampling_rate):
    words = list()
    
    fname= './data/namu_wonnie.txt'
    
    with open(fname, 'r') as f:
        namu = [data.split() for data in f]
        
    print(len(namu))
    
    for data in namu:
        words.append(data)
        
    for line in desc_list:
        sentence = re.sub(r"[^ㄱ-힣a-zA-Z0-9]+", ' ', line).strip().split()
        if sentence:
            words.append(sentence)

    word_counter = [['UNK', -1]] #시작점
    word_counter.extend(collections.Counter([word for sentence in words for word in sentence]).most_common())
    word_counter = [item for item in word_counter if item[1] >= min_count or item[0] == 'UNK']

    word_list = list()
    word_dict = dict()
    for word, count in word_counter:
        word_list.append(word) # 학습에 사용된 word를 저장한다. (visualize를 위해)
        word_dict[word] = len(word_dict)
    word_reverse_dict = dict(zip(word_dict.values(), word_dict.keys()))

    word_to_pos_li = dict()
    pos_list = list()
    twitter = Twitter()
    for w in word_dict:
        w_pos_li = list()
        for pos in twitter.pos(w, norm=True):
            w_pos_li.append(pos)

        word_to_pos_li[word_dict[w]] = w_pos_li
        pos_list += w_pos_li

    pos_counter = collections.Counter(pos_list).most_common()

    pos_dict = dict()
    for pos, _ in pos_counter:
        pos_dict[pos] = len(pos_dict)

    pos_reverse_dict = dict(zip(pos_dict.values(), pos_dict.keys()))

    word_to_pos_dict = dict()

    for word_id, pos_li in word_to_pos_li.items():
        pos_id_li = list()
        for pos in pos_li:
            pos_id_li.append(pos_dict[pos])
        word_to_pos_dict[word_id] = pos_id_li

    data = list()
    unk_count = 0
    for sentence in words:
        s = list()
        for word in sentence:
            if word in word_dict:
                index = word_dict[word]
            else:
                index = word_dict['UNK']
                unk_count += 1
            s.append(index)
        data.append(s)
    word_counter[0][1] = max(1, unk_count)

    # data = sub_sampling(data, word_counter, word_dict, sampling_rate)

    return data, word_dict, word_reverse_dict, pos_dict, pos_reverse_dict, word_to_pos_dict, word_list

def sub_sampling(data, word_counter, word_dict, sampling_rate):
    total_words = sum([len(sentence) for sentence in data])
    # print("total_words: {}".format(total_words))
    prob_dict = dict()
    for word, count in word_counter:
        f = count / total_words # 빈도수가 많을수록 f가 1에 가까워짐.
        p = max(0, 1 - math.sqrt(sampling_rate / f)) # sampling_rate가 0.0001이면 f가 클수록 prob이 커진다.
        prob_dict[word_dict[word]] = p
        # print("count : {}, f : {}, p : {}, prob_dict : {}".format(count, f, p, prob_dict))

    new_data = list()
    for sentence in data:
        s = list()
        for word in sentence:
            prob = prob_dict[word]
            if random.random() > prob: # prob이 작을수록 s에 저장되기 쉬움.
                s.append(word)
        new_data.append(s)

    return new_data

In [3]:
# crawling한 데이터를 불러온다.
pk_data = pd.read_csv(DATA_PATH + 'pk_data_g1.csv')
desc_list = []
for i in range(len(pk_data)):
    for desc in pk_data['desc'][i].split('.'):
        desc_list.append(desc)

sampling_rate = 0.0001
min_count = 5

data, word_dict, word_reverse_dict, pos_dict, pos_reverse_dict, word_to_pos_dict, word_list \
        = build_dataset(desc_list, min_count, sampling_rate)

460448


In [4]:
# 학습에 사용된 word list 저장
f = open("word_list.txt", 'w')
for word in word_list:
    input_word = "{} ".format(word)
    f.write(input_word)
f.close()

In [5]:
vocabulary_size = len(word_dict)
pos_size = len(pos_dict)
num_sentences = len(data)

print("number of sentences :", num_sentences)
print("vocabulary size :", vocabulary_size)
print("pos size :", pos_size)

pos_li = []
for key in sorted(pos_reverse_dict):
    pos_li.append(pos_reverse_dict[key])

number of sentences : 465247
vocabulary size : 94749
pos size : 33304


In [6]:
window_size = 5
batch_size = 150

# kor2vec 의 input index list와 output index list를 만든다.
# 윈도우 사이즈에 따라 input output pair가 늘어난다.(input이 중복)
def generate_input_output_list(data, window_size):
    input_li = list()
    output_li = list()
    for sentence in data:
        for i in range(len(sentence)):
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i != j:
                    if sentence[i]!=word_dict['UNK'] and sentence[j]!=word_dict['UNK']:
                        input_li.append(sentence[i])
                        output_li.append(sentence[j])
    return input_li, output_li

input_li, output_li = generate_input_output_list(data, window_size)
input_li_size = len(input_li)

In [7]:
print(batch_size)
print(input_li_size)
def generate_batch(iter, batch_size, input_li, output_li):
    index = (iter % (input_li_size//batch_size)) * batch_size
    batch_input = input_li[index:index+batch_size]
    batch_output_li = output_li[index:index+batch_size]
    batch_output = [[i] for i in batch_output_li]

    return np.array(batch_input), np.array(batch_output)

batch_inputs, batch_labels = generate_batch(0, batch_size, input_li, output_li)
print(np.shape(batch_inputs))
print(batch_inputs)
print(np.shape(batch_labels))
print(batch_labels)
word_list = []
for word in batch_inputs:
    word_list.append(word_to_pos_dict[word])
print(word_list)
#     for pos in word_to_pos_dict[word]:
#         print(pos)
#         print(pos_reverse_dict[pos])

150
20073212
(150,)
[ 1969  2533     1     1     1  6181  6181  6181     8     8     8 39568
 39568 39568 27947 27947  1732  1732 22535 22535    54 59115    54 67444
    54 18937 39568 39568 39568 39568 39568 43173 43173 43173 43173 43173
 59116 59116 59116 59116 59116 59116 67445 67445 67445 67445 67445 67445
  4149  4149  4149  4149  4149  4149  4149  4536  4536  4536  4536  4536
  4536  4536  4199  4199  4199  4199  4199 36490 36490 36490 36490 36490
  2175  2175  2175  2175 12893 12893 12893 12893    21    21    21    21
 21477 21477 21477 21477   799   799   799   799   799  1642  1642  1642
  1642  1642   251   251   251   251   251   251  3954  3954  3954  3954
  3954  3954  3954  7538  7538  7538  7538  7538  7538  7538 67446 67446
 67446 67446 67446 67446 67446 67446 18201 18201 18201 18201 18201 18201
 18201 18201 10157 10157 10157 10157 10157 10157 10157 10157 10157 15119
 15119 15119 15119 15119 15119 15119]
(150, 1)
[[ 2533]
 [ 1969]
 [ 6181]
 [    8]
 [39568]
 [    1]
 [ 

In [8]:
embedding_size = 150
num_sampled = 50
learning_rate = 1.0

valid_size = 20     # Random set of words to evaluate similarity on.
valid_window = 200  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False) # 200까지 숫자 중에서 랜덤하게 20개 뽑음

# tensorflow 신경망 모델 그래프 생성
graph = tf.Graph()
with graph.as_default():
    # Input data
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    words_matrix = [tf.placeholder(tf.int32, shape=None) for _ in range(batch_size)] # batch_size만큼의 word를 형태소로
    vocabulary_matrix = [tf.placeholder(tf.int32, shape=None) for _ in range(vocabulary_size)] # word_dict만큼의 word를 형태소로.. 인거 같은데 안씀
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # "/device:GPU:0"
    with tf.device('/cpu:0'):
        # embedding vector -> 우리가 원하는 최종 출력
        pos_embeddings = tf.Variable(tf.random_uniform([pos_size, embedding_size], -1.0, 1.0), name='pos_embeddings')

        word_vec_list = []
        for i in range(batch_size):
            word_vec = tf.reduce_sum(tf.nn.embedding_lookup(pos_embeddings, words_matrix[i]), 0)
            word_vec_list.append(word_vec)
        word_embeddings = tf.stack(word_vec_list) # word의 각 형태소를 embedding한 vector
    
        # Noise-Contrastive Estimation
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)), name='nce_weights'
        )
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]), name='nce_biases')

    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=word_embeddings,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    init = tf.global_variables_initializer()

    # Compute the cosine similarity between minibatch exaples and all embeddings.
    # 임의의 word로 유사도 검증
    norm = tf.sqrt(tf.reduce_sum(tf.square(pos_embeddings), 1, keep_dims=True))
    normalized_embeddings = pos_embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
num_iterations = input_li_size // batch_size
print("number of iterations for each epoch :", num_iterations)
epochs = 4
num_steps = num_iterations * epochs + 1

with tf.Session(graph=graph) as session:
    init.run()
    print("Initialized - Tensorflow")

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(step, batch_size, input_li, output_li)

        word_list = []
        for word in batch_inputs:
            word_list.append(word_to_pos_dict[word])

        feed_dict = {}
        for i in range(batch_size):
            feed_dict[words_matrix[i]] = word_list[i]
        feed_dict[train_inputs] = batch_inputs
        feed_dict[train_labels] = batch_labels

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % (num_steps//10) == 0:
            if step > 0:
                average_loss /= 2000
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        if step % (num_steps//4) == 0:
            pos_embed = pos_embeddings.eval()

            # Print nearest words
            sim = similarity.eval()
            for i in range(valid_size):
                valid_pos = pos_reverse_dict[valid_examples[i]]
                top_k = 8
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % str(valid_pos)
                for k in range(top_k):
                    close_word = pos_reverse_dict[nearest[k]]
                    log_str = '%s %s,' % (log_str, str(close_word))
                print(log_str)

    pos_embed = pos_embeddings.eval()

number of iterations for each epoch : 133821
Initialized - Tensorflow
Average loss at step  0 :  216.94976806640625
Nearest to ('에서', 'Noun'): ('넘겨주기', 'Noun'), ('하자', 'Verb'), ('어야만', 'Eomi'), ('경제성장률', 'Noun'), ('준우승', 'Noun'), ('베이가', 'Noun'), ('HANDS', 'Alpha'), ('싱크', 'Noun'),
Nearest to ('키', 'Noun'): ('봉화', 'Noun'), ('담겨', 'Verb'), ('트루먼', 'Noun'), ('넓다', 'Adjective'), ('조정하는', 'Verb'), ('러시', 'Noun'), ('안정감', 'Noun'), ('해주는', 'Verb'),
Nearest to ('에서는', 'Josa'): ('내일', 'Noun'), ('Adventure', 'Alpha'), ('실현되', 'Verb'), ('9000', 'Number'), ('엑스트라', 'Noun'), ('컴백', 'Noun'), ('제독', 'Noun'), ('생각되는', 'Verb'),
Nearest to ('이었', 'Verb'): ('착하면', 'Adjective'), ('아테네', 'Noun'), ('커진', 'Verb'), ('소규모', 'Noun'), ('컵', 'Noun'), ('1813', 'Number'), ('온앤', 'Noun'), ('메모리즈', 'Noun'),
Nearest to ('이', 'Determiner'): ('고시', 'Noun'), ('초록색', 'Noun'), ('흑사병', 'Noun'), ('Flower', 'Alpha'), ('맞춰', 'Verb'), ('내려온', 'Verb'), ('아르카디아', 'Noun'), ('이끌었', 'Verb'),
Nearest to ('들', 'Suffix'): ('NCAA', 'Al

In [ ]:
# Function to save vectors.
def save_model(pos_list, embeddings, file_name):
    with open(file_name, 'w') as f:
        f.write(str(len(pos_list)))
        f.write(" ")
        f.write(str(embedding_size))
        f.write("\n")
        for i in range(len(pos_list)):
            pos = pos_list[i]
            f.write(str(pos).replace("', '", "','") + " ")
            f.write(' '.join(map(str, embeddings[i])))
            f.write("\n")

# Save vectors
save_model(pos_li, pos_embed, "pos.vec")